In [5]:
import torch
from torchvision import transforms, models
from transformers import CLIPProcessor, CLIPModel, BlipProcessor, BlipForConditionalGeneration
from diffusers import StableDiffusionPipeline, DDIMScheduler
from torch.nn import functional as F
import numpy as np

# Device Configuration
device = "cuda" if torch.cuda.is_available() else "cpu"

# 1. BLIP: Image Captioning
class ImageCaptioning:
    def __init__(self):
        self.processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        self.model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

    def get_caption(self, image):
        inputs = self.processor(image, return_tensors="pt").to(device)
        caption_ids = self.model.generate(**inputs)
        caption = self.processor.decode(caption_ids[0], skip_special_tokens=True)
        return caption


# 2. CLIP: Text Embedding
class TextEmbedding:
    def __init__(self):
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        self.model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

    def get_embedding(self, caption):
        inputs = self.processor(text=[caption], return_tensors="pt", padding=True).to(device)
        text_embedding = self.model.get_text_features(**inputs)
        return text_embedding


# 3. Stable Diffusion Feature Extraction
class FeatureExtractor:
    def __init__(self, model_name="runwayml/stable-diffusion-v1-5"):
        self.pipe = StableDiffusionPipeline.from_pretrained(model_name).to(device)
        # The scheduler configuration is likely located under the model's directory.
        # Specify the 'scheduler' subfolder:
        self.scheduler = DDIMScheduler.from_pretrained(model_name, subfolder="scheduler")
        # Call set_timesteps to initialize num_inference_steps
        self.scheduler.set_timesteps(50) # You can adjust the number of steps here

    def extract_features(self, image, text_embedding):
        # Encode the image to latent space
        latents = self.pipe.vae.encode(image.unsqueeze(0).to(device)).latent_dist.sample()
        latents = latents * self.pipe.vae.config.scaling_factor

        # Invert using DDIM
        noise = torch.randn_like(latents).to(device)
        inverted_latents = self.scheduler.add_noise(latents, noise, torch.tensor([49], device=device, dtype=torch.long)) # Passing a fixed timestep value

        # Reconstruct image from inverted latent
        with torch.no_grad():
            # Passing noise and inverted_latents as arguments and removing text_embedding as it is not the timestep
            reconstructed_latents = self.scheduler.step(noise, 49, inverted_latents).prev_sample
            reconstructed_image = self.pipe.vae.decode(reconstructed_latents / self.pipe.vae.config.scaling_factor).sample

        return latents, noise, reconstructed_image


# 4. Classifier
class FakeImageClassifier(torch.nn.Module):
    def __init__(self):
        super(FakeImageClassifier, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = torch.nn.Linear(self.resnet.fc.in_features, 2)  # Binary classification

    def forward(self, inputs):
        return self.resnet(inputs)



# 5. Detection Pipeline
# 5. Detection Pipeline
class FakeInversionPipeline:
    def __init__(self):
        self.captioning = ImageCaptioning()
        self.embedding = TextEmbedding()
        self.feature_extractor = FeatureExtractor()
        self.classifier = FakeImageClassifier().to(device)

    def detect(self, image):
        # Step 1: Generate caption
        caption = self.captioning.get_caption(image)

        # Step 2: Get text embedding
        text_embedding = self.embedding.get_embedding(caption)

        # Step 3: Extract features
        latent, noise, reconstructed_image = self.feature_extractor.extract_features(image, text_embedding)

        # Step 3: Extract features
        reconstructed_image = reconstructed_image.squeeze(0) # Remove batch dimension if present

        # Pass reconstructed_image directly to the classifier
        # Add an extra dimension for the batch size
        output = self.classifier(reconstructed_image.unsqueeze(0))
        prediction = torch.argmax(F.softmax(output, dim=1), dim=1)

        return "Fake" if prediction.item() == 1 else "Real"\
# Example Usage
if __name__ == "__main__":
    from PIL import Image

    # Load image
    image_path = "contracts-analysis.png"  # Replace with your image path
    image = Image.open(image_path).convert("RGB")
    preprocess = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor()
    ])
    # Removed one unsqueeze call, it is added already inside the extract_features method
    image = preprocess(image).to(device) #  .unsqueeze(0)

    # Initialize pipeline
    pipeline = FakeInversionPipeline()
    result = pipeline.detect(image)
    print(f"Detection Result: {result}")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Detection Result: Fake
